In [ ]:
import param
import pandas as pd
import panel as pn

You can use the `WebComponent` to quickly **plugin webcomponents or Javascript libraries** not already supported in Panel. 

So if you are not satisfied with the look and feel of the existing Panel widgets then use the `WebComponent` to plug in your favourite set of widgets. Or if the `DataFrame` pane or widget is not enough for your use case plugin an alternative data table.

For an introduction to *web components* see [Web Components: the secret ingredient helping Power the web](https://www.youtube.com/watch?v=YBwgkr_Sbx0).

<a href="https://www.youtube.com/watch?v=YBwgkr_Sbx0" target="blank_"><img src="https://i.ytimg.com/vi/YBwgkr_Sbx0/hqdefault.jpg"></img></a>

Please note that picking and using a web component library can be challenging as the **web component tools, frameworks and standard is rapidly evolving**. The newest and best implemented web components will be easiest to use. The `WebComponent` it self is a also a new feature in Panel (Q2 2020). So we need your help to identify bugs and improvements, for the time being the component is marked as **experimental**.

Parameters
----------

You can use the `WebComponent` by instantiating an instance or inheriting from it.

- `html`: The web component html tag.
    - For example `<mwc-button></mwc-button>`. But can be more complex.
- `attributes_to_watch`: A dictionary of (`html_attribute`, `python_parameter`) names
    - The value of `python_parameter`  will be used to set the `html_attribute` on construction. 
    - The value of `python_parameter` and `html_attribute` will be kept in sync (*two way binding*).
    - The value of `html_attribute` may not be None
    - The value of `python_parameter` can be None.
- `properties_to_watch`: A dictionary of (`js_property`, `python_parameter`) names
    - The value of `python_parameter`  will be used to set the `js_property` on construction. 
    - The value of `python_parameter` and `js_property` will be kept in sync (*two way binding*).
    - The value of `js_property` may not be None
    - The value of `python_parameter` can be None.
    - You can specify a nested `js_property` like `textInput.value` as key.
- `parameters_to_watch`: Can be used to make the `html` parameter value dynamic. The list of `parameters_to_watch` will be watched and when changed the `html` will be updated. You need to implement the `_get_html_from_parameters_to_watch` to return the updated `html` value.
- `events_to_watch`: A Dictionary of (`js_event`, `python_parameter_to_inc`) names.
    - The `js_event` will be watched on the JavaScript side. When fired the javascript code will 
        - Increment the `python_parameter_to_inc` with `+1` if specified.
        - check whether any `js_property` key in `properties_to_watch` has changed and if yes then it will be synced to the associated `python_parameter` value.
- The `column_data_source` can be used to efficiently transfer columnar data to the javascript side.
    - The `column_data_source_orient` is used to specify how the data should be input to the `column_data_source_load_function` below.
         - For now `dict` and `records` are supported.
         - See the `orient` parameter of the [pandas.to_dict](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html) function for more info.
- The `column_data_source_load_function` specifies the name of the js function or property that will load the data.
- If the JavaScript WebComponent contains an `after_layout` function this is used to
resize the JS WebComponent. See the `ECharts` web component in the Gallery for an example.

You will propably have to experiment a bit in order to determine which javascript files to import and what combination of attributes, properties, events and/ or
parameters to watch.
___

In order for WebComponents to function we need to load the corresponding Javascript modules. Below we load the MWC button and MWC slider components along with Material Icons CSS. Note that the webcomponent-loader.js is only required for older browsers which do not yet support Webcomponents natively.

In [ ]:
js_urls = {
    'webcomponent_loader': 'https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/2.4.3/webcomponents-loader.js',
    'button': 'https://www.unpkg.com/@material/mwc-button?module',
    'slider': 'https://www.unpkg.com/@material/mwc-slider?module',
    'lit_table': 'https://unpkg.com/@doubletrade/lit-datatable@0.3.5/lit-datatable.js?module'
}

css_urls = [
    'https://fonts.googleapis.com/css?family=Roboto:300,400,500',
    'https://fonts.googleapis.com/css?family=Material+Icons&display=block'
]

css = """
:root {
    --mdc-theme-primary: green;
    --mdc-theme-secondary: purple;
}
"""

pn.extension(js_files=js_urls, css_files=css_urls, raw_css=[css])

### `attributes_to_watch`

Lets create a `MWCButton`.

In [ ]:
MWC_ICONS = [None, "accessibility", "code", "favorite"] # For more icons see https://material.io/resources/icons/?style=baseline

class MWCButton(pn.pane.WebComponent):

    html = param.String("<mwc-button></mwc-button")

    attributes_to_watch = param.Dict({"label": "name", "icon": "icon", "raised":"raised"})
    
    raised=param.Boolean(default=True)

    icon=param.ObjectSelector(default="favorite", objects=MWC_ICONS)
    
    height = param.Integer(default=30)

mwc_button = MWCButton(name="Panel")
mwc_button

Lets play with the attributes

In [ ]:
pn.Param(
    mwc_button, parameters=["name", "icon", "raised", "height"]
)

For an example that use the `attributes_to_watch` bi-directionally take a look at the `perspective-viewer` example in the Gallery.

### `properties_to_watch`

Lets create a `MWCSlider`

In [ ]:
mwc_slider_html = """
<mwc-slider
    step="5"
    pin
    markers
    max="50"
    value="10">
</mwc-slider>
"""

class MWCSlider(pn.pane.WebComponent):
    html = param.String(mwc_slider_html)
    properties_to_watch = param.Dict({"value": "value"})
    
    value = param.Integer(default=10, bounds=(0,50), step=5)
    height= param.Integer(default=50)

    
mwc_slider = MWCSlider(margin=(20,10,0,10))
mwc_slider

In [ ]:
pn.Param(mwc_slider, parameters=["value"])

### `events_to_Watch`

Lets add `clicks` count to the `MWCButton`

In [ ]:
MWC_ICONS = [None, "accessibility", "code", "favorite"] # For more icons see https://material.io/resources/icons/?style=baseline

class MWCButton(pn.pane.WebComponent):
    html = param.String("<mwc-button></mwc-button")
    attributes_to_watch = param.Dict({"label": "name", "icon": "icon", "raised":"raised"})
    
    raised=param.Boolean(default=True)
    icon=param.ObjectSelector(default="favorite", objects=MWC_ICONS, allow_None=True)
    
    height = param.Integer(default=30)
    
    # NEW IN THIS EXAMPLE
    events_to_watch = param.Dict({"click": "clicks"})
    clicks = param.Integer()
    
    

mwc_button = MWCButton(name="Panel")
mwc_button

In [ ]:
pn.Param(
    mwc_button, parameters=["name", "icon", "raised", "height", "clicks"]
)

## Lit-DataTable Example

In this example we will use the [lit-datatable](https://github.com/DoubleTrade/lit-datatable) which is a material design implementation of a data table.

### `properties_to_watch`

Above we used the `html` attributes `data` and `conf`. So we can build a version of `LitDataTable` based on `attributes_to_watch`.

But the the `data` and `conf` `html` attributes also corresponds to `data` and `conf` properties on the `js` object. It is not always the case for web components that there is a 1-1 correspondance though.

Lets build the `LitDataTable` based on `properties_to_watch` to illustrate this.

In [ ]:
class LitDataTable1(pn.pane.WebComponent):
    html = param.String("<lit-datatable><lit-datatable>")
    properties_to_watch = param.Dict({"data": "data", "conf": "conf"})
    
    data = param.List()
    conf = param.List()

In [ ]:
data = [
  { "fruit": "apple", "color": "green", "weight": "100gr" },
  { "fruit": "banana", "color": "yellow", "weight": "140gr" }
]
conf = [
  { "property": "fruit", "header": "Fruit", "hidden": False },
  { "property": "color", "header": "Color", "hidden": False },
  { "property": "weight", "header": "Weight", "hidden": False }
]

lit_data_table1 = LitDataTable1(conf=conf, data=data, height=150)
lit_data_table1

### `column_data_source`

If we wanted to transfer a `DataFrame` and/ or large amounts of data to the `lit-datatable` we would create version of `LitDataTable` using `column-data_source`. Let's do that.

In [ ]:
class LitDataTable(pn.pane.WebComponent):
    html = param.String("<lit-datatable><lit-datatable>")
    properties_to_watch = param.Dict({"conf": "conf"})
 
    conf = param.List()
    
    column_data_source_orient = param.String("records")
    column_data_source_load_function = param.String("data")

In [ ]:
import pandas as pd
from bokeh.models import ColumnDataSource

dataframe = pd.DataFrame(data)

column_data_source = ColumnDataSource(dataframe)

lit_data_table = LitDataTable(conf=conf, column_data_source=column_data_source, height=150)
lit_data_table

Lets a replace the data and see that it updates

In [ ]:
new_data = [
    { "fruit": "apple", "color": "green", "weight": "100gr" },
    { "fruit": "banana", "color": "yellow", "weight": "140gr" },
    { "fruit": "pineapple", "color": "yellow", "weight": "1000gr" },
]
new_conf = [
  { "property": "fruit", "header": "Fruit (name)", "hidden": False },
  { "property": "color", "header": "Color", "hidden": False },
  { "property": "weight", "header": "Weight (g)", "hidden": False }
]
new_dataframe = pd.DataFrame(new_data)

lit_data_table.column_data_source.data = ColumnDataSource.from_df(new_dataframe)
lit_data_table.height = 200
lit_data_table.conf = new_conf

## Web Component Libraries

You can find web components at

- [Awesome Lit Element components](https://github.com/web-padawan/awesome-lit-html#components)
- [webcomponents.org](https://www.webcomponents.org/)
- [npm](https://www.npmjs.com/)

Some example libraries are

- [Amber](https://amber.bitrock.it/components/overview/)
- [Material](https://github.com/material-components/material-components-web-components#readme), [Demo](https://mwc-demos.glitch.me/demos/)
- [Microsoft Graph Toolkit](https://github.com/microsoftgraph/microsoft-graph-toolkit)
- [SAP UI5](https://github.com/SAP/ui5-webcomponents), [Demo](https://sap.github.io/ui5-webcomponents/playground)
- [Smart Elements](https://www.webcomponents.org/element/@smarthtmlelements/smart-bootstrap)

### Prebuilt libraries from unpkg.com

If you find a web component library on [npm](https://npmjs.com) you can find the corresponding precombiled library on [unpkg](https://unpkg.com) for use with Panel.

For example if you find the `wired-button` at [https://www.npmjs.com/package/wired-button](https://www.npmjs.com/package/wired-button) then you can browse the precompiled files at [https://www.unpkg.com/browse/wired-button/](https://www.unpkg.com/browse/wired-button/) to locate the relevant precombiled file at [https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js](https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js).

## References

- [Build an app with WebComponents in 9 minutes](https://www.youtube.com/watch?v=mTNdTcwK3MM)
- [How to use Web Components in a JavaScript project](https://www.youtube.com/watch?v=88Sa-SlHRxk&t=63s)